In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import nltk, warnings
import itertools
from statistics import mean
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import LocalOutlierFactor
from numpy import where
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
p1=pd.read_csv('/content/drive/MyDrive/3clusters_38_allweeks.csv')
p1.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,client_id,name38,name39,name40,name41,name42,name43,name44,name45,...,name81,name82,name83,name84,name85,name86,name87,name88,name89,name90
0,0,0005586058dff230bd98fc572cf3ad1b,sleeping,NaN,NaN,sleeping,NaN,NaN,sleeping,NaN,...,NaN,NaN,NaN,NaN,NaN,sleeping,NaN,NaN,NaN,sleeping
1,1,000a84ecdc62cb41a42327a5bf307221,loyal,champions,sleeping,sleeping,sleeping,NaN,sleeping,NaN,...,sleeping,sleeping,champions,NaN,NaN,sleeping,loyal,sleeping,NaN,sleeping
2,2,001668f279a5798e46c0d79050004b72,NaN,NaN,NaN,sleeping,NaN,NaN,NaN,NaN,...,NaN,sleeping,NaN,sleeping,loyal,NaN,sleeping,NaN,sleeping,sleeping
3,3,001c54509cd864d038f83f9efbd5696d,champions,loyal,champions,sleeping,loyal,sleeping,loyal,NaN,...,champions,sleeping,champions,sleeping,loyal,champions,sleeping,loyal,sleeping,sleeping
4,4,001da94244bab7dd0ea8bd5d1ef148a0,champions,NaN,sleeping,sleeping,NaN,NaN,sleeping,champions,...,loyal,sleeping,loyal,sleeping,NaN,loyal,NaN,sleeping,sleeping,NaN


In [3]:
df = p1.loc[:, p1.columns != 'Unnamed: 0']
df = df.fillna('churn')
df_r=df.replace('churn','4: churn').replace('champions','3: champions').replace('loyal','2: loyal').replace('sleeping','1: sleeping')

df_r.columns=df_r.columns.str.replace('name','period')
df_r.head()

,client_id,period38,period39,period40,period41,period42,period43,period44,period45,period46,...,period81,period82,period83,period84,period85,period86,period87,period88,period89,period90
0,0005586058dff230bd98fc572cf3ad1b,1: sleeping,4: churn,4: churn,1: sleeping,4: churn,4: churn,1: sleeping,4: churn,4: churn,...,4: churn,4: churn,4: churn,4: churn,4: churn,1: sleeping,4: churn,4: churn,4: churn,1: sleeping
1,000a84ecdc62cb41a42327a5bf307221,2: loyal,3: champions,1: sleeping,1: sleeping,1: sleeping,4: churn,1: sleeping,4: churn,4: churn,...,1: sleeping,1: sleeping,3: champions,4: churn,4: churn,1: sleeping,2: loyal,1: sleeping,4: churn,1: sleeping
2,001668f279a5798e46c0d79050004b72,4: churn,4: churn,4: churn,1: sleeping,4: churn,4: churn,4: churn,4: churn,4: churn,...,4: churn,1: sleeping,4: churn,1: sleeping,2: loyal,4: churn,1: sleeping,4: churn,1: sleeping,1: sleeping
3,001c54509cd864d038f83f9efbd5696d,3: champions,2: loyal,3: champions,1: sleeping,2: loyal,1: sleeping,2: loyal,4: churn,1: sleeping,...,3: champions,1: sleeping,3: champions,1: sleeping,2: loyal,3: champions,1: sleeping,2: loyal,1: sleeping,1: sleeping
4,001da94244bab7dd0ea8bd5d1ef148a0,3: champions,4: churn,1: sleeping,1: sleeping,4: churn,4: churn,1: sleeping,3: champions,4: churn,...,2: loyal,1: sleeping,2: loyal,1: sleeping,4: churn,2: loyal,4: churn,1: sleeping,1: sleeping,4: churn


In [4]:
a=df_r[['period38', 'client_id']].groupby('period38').count()
a1=a.reset_index()
a1.columns=['column', 'count38']
for column in df_r.columns.to_list()[2:]:
  aa=df_r[[column, 'client_id']].groupby(column).count()
  aaa=aa.reset_index()
  aaa.columns=['column', column]
  a1=a1.merge(aaa, on='column', how='left')
a1

,column,count38,period39,period40,period41,period42,period43,period44,period45,period46,...,period81,period82,period83,period84,period85,period86,period87,period88,period89,period90
0,1: sleeping,7180,15318,13537,13343,13331,13523,13617,14227,14189,...,13806,13665,13048,13193,13768,13566,12898,13888,12538,12873
1,2: loyal,2462,2511,2615,2567,2490,2573,2619,2637,2793,...,2839,2770,2656,2589,2589,2627,2427,2807,2306,2274
2,3: champions,1667,1390,2079,2419,2167,2271,2342,2424,2313,...,1249,1175,1089,1168,1170,1139,2650,1296,2612,2601
3,4: churn,73593,65683,66671,66573,66914,66535,66324,65614,65607,...,67008,67292,68109,67952,67375,67570,66927,66911,67446,67154


In [5]:
a2=a1.T
a2.head()


,0,1,2,3
column,1: sleeping,2: loyal,3: champions,4: churn
count38,7180,2462,1667,73593
period39,15318,2511,1390,65683
period40,13537,2615,2079,66671
period41,13343,2567,2419,66573


In [6]:
a2=a2.iloc[1:,0:]
a2.columns=['sleeping', 'loyal',	'champions', 'churn']
a2

,sleeping,loyal,champions,churn
count38,7180,2462,1667,73593
period39,15318,2511,1390,65683
period40,13537,2615,2079,66671
period41,13343,2567,2419,66573
period42,13331,2490,2167,66914
period43,13523,2573,2271,66535
period44,13617,2619,2342,66324
period45,14227,2637,2424,65614
period46,14189,2793,2313,65607
period47,13890,2740,2404,65868


In [7]:
a2.to_csv('count_of_clusters_38.csv')

In [ ]:

for stroka in [0,1,2,3]:
  col=0
  spisok=[]
  for period in range(38,90):

    df2=df_r[['client_id', f'period{str(period)}', f'period{str(period+1)}']]
    output = df2.pivot_table(index=f'period{str(period+1)}', columns=f'period{str(period)}', values='client_id', aggfunc='count', margins=True,margins_name='Grand Total')
    for i in range(4):
      output.iloc[int(i),:]=round((output.iloc[int(i),:]/output.iloc[4,:])*100,1)
      outputt=output.iloc[:4, :4]
  
    spisok.append(outputt.iloc[stroka,3])
  print(str(spisok))

[15.0, 11.3, 11.3, 11.3, 11.5, 11.8, 12.5, 12.1, 11.8, 12.3, 12.4, 13.6, 14.1, 17.3, 13.7, 12.2, 12.2, 11.7, 11.7, 11.8, 12.5, 13.8, 11.9, 15.0, 11.7, 13.1, 10.5, 7.6, 9.0, 9.4, 8.4, 9.6, 9.5, 9.6, 10.2, 8.9, 10.9, 11.5, 11.5, 11.5, 11.5, 11.3, 11.5, 11.4, 10.8, 11.0, 11.5, 11.3, 11.5, 11.6, 10.5, 11.0]
[2.3, 1.0, 1.0, 0.9, 0.9, 1.0, 1.0, 1.1, 0.9, 1.0, 0.9, 1.1, 1.3, 3.8, 1.3, 0.9, 1.0, 0.9, 0.9, 2.0, 1.1, 0.4, 0.9, 1.6, 0.9, 2.1, 2.1, 1.8, 0.8, 2.0, 0.7, 2.2, 1.9, 1.9, 2.1, 2.8, 2.0, 2.2, 2.1, 1.9, 2.0, 2.1, 2.1, 2.1, 2.0, 2.0, 2.1, 2.0, 1.0, 2.2, 0.9, 0.9]
[0.7, 1.5, 1.9, 1.6, 1.8, 1.8, 1.9, 1.8, 1.8, 1.9, 1.7, 2.4, 2.5, 0.7, 2.3, 1.7, 1.9, 1.7, 1.9, 0.3, 2.0, 2.1, 1.9, 2.2, 1.8, 0.4, 0.3, 0.2, 1.9, 0.4, 1.7, 0.3, 0.3, 0.4, 0.3, 0.9, 0.3, 0.3, 0.4, 0.4, 0.3, 0.3, 0.4, 0.3, 0.3, 0.3, 0.4, 0.4, 2.1, 0.4, 2.1, 2.1]
[82.0, 86.2, 85.9, 86.1, 85.8, 85.4, 84.6, 85.0, 85.5, 84.8, 84.9, 82.9, 82.2, 78.2, 82.7, 85.1, 84.9, 85.7, 85.6, 85.9, 84.4, 83.7, 85.3, 81.1, 85.6, 84.4, 87.1, 90.4, 88.3